In [ ]:
import os
os.environ['KAGGLE_USERNAME']="simransohal"
os.environ['KAGGLE_KEY']="3aa7697cd9a4ba2a3715c4a29c0f0cb6"

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
ls

asl_alphabet_test/  asl_alphabet_train/  asl-alphabet.zip  sample_data/


In [ ]:
import zipfile
with zipfile.ZipFile('asl-alphabet.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
ls

asl_alphabet_test/  asl_alphabet_train/  asl-alphabet.zip  sample_data/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.optim import Adam

In [ ]:
d=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model=models.resnet18(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
ls

asl_alphabet_test/  asl_alphabet_train/  asl-alphabet.zip  sample_data/


In [ ]:
#Getting the number of input features in fully connected layer
#Model is to be classified in just three layers
#Created a new layer and replaced the fully connected layer with it
num_classes=3
features=model.fc.in_features
model.fc=nn.Linear(features,num_classes)

In [ ]:
model=model.to(d)

In [ ]:
transformations=transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])

In [ ]:
data=datasets.ImageFolder(r"asl_alphabet_train",transform=transformations)

In [ ]:
test_data=datasets.ImageFolder(r"asl_alphabet_test",transform=transformations)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [ ]:
from torch.utils.data import DataLoader
loader_train=DataLoader(data,batch_size=128,shuffle=True )
loader_test=DataLoader(data,batch_size=128,shuffle=True)

In [ ]:
n=5
for epoch in range(n):
  #Going through the batches of train images
  for images, labels in loader_train:
    #transfering labels and images to gpu
    images,labels=images.to(d),labels.to(d)
    #resets gradient of all batches, because by default the gradients are accumulated 
    optimizer.zero_grad()
    #passes images through model to obtain predicted output
    result=model(images)
    #predicting loss
    loss=criterion(result,labels)
    #computes gradient of loss
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{n}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 1.1570
Epoch [1/5], Loss: 0.7272
Epoch [1/5], Loss: 0.5263
Epoch [1/5], Loss: 0.2458
Epoch [1/5], Loss: 0.1857
Epoch [1/5], Loss: 0.0303
Epoch [1/5], Loss: 0.0136
Epoch [1/5], Loss: 0.0061
Epoch [1/5], Loss: 0.0039
Epoch [1/5], Loss: 0.0027
Epoch [1/5], Loss: 0.0020
Epoch [1/5], Loss: 0.0023
Epoch [1/5], Loss: 0.0015
Epoch [1/5], Loss: 0.0012
Epoch [1/5], Loss: 0.0015
Epoch [1/5], Loss: 0.0010
Epoch [1/5], Loss: 0.0008
Epoch [1/5], Loss: 0.0004
Epoch [1/5], Loss: 0.0004
Epoch [1/5], Loss: 0.0003
Epoch [1/5], Loss: 0.0003
Epoch [1/5], Loss: 0.0003
Epoch [1/5], Loss: 0.0002
Epoch [1/5], Loss: 0.0002
Epoch [1/5], Loss: 0.0002
Epoch [1/5], Loss: 0.0002
Epoch [1/5], Loss: 0.0002
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5], Loss: 0.0001
Epoch [1/5],

In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total=0
  for image,label in loader_test:
    images,label=images.to(d),labels.to(d)
    result=model(images)
    _,predicted=torch.max(result.data,1)
    total=total+labels.size(0)
    correct=correct+(predicted==labels).sum().items()
    print(f'Accuracy {100 * correct / total:.2f}%')


